In [1]:
import torch
import numpy as np
from MVCNN.models.MVCNN import SVCNN
from MVCNN_Trainer import MVCNN_Trainer
from PruningFineTuner import PruningFineTuner
from Pruning import Pruning
from FilterPruner import FilterPruner

In [2]:
model = SVCNN('svcnn')
model.load_state_dict(torch.load('./model-00030.pth', map_location='mps'))
model = model.to('mps')

In [3]:
mvcnntrainer = MVCNN_Trainer(optimizer=torch.optim.Adam(model.parameters(), lr=0.01))

In [4]:
mvcnntrainer.get_size(model), mvcnntrainer.get_num_filters(model)

(491.72034072875977, 2752)

In [5]:
pft = PruningFineTuner(model, train_amt=0.01, test_amt=0.1)

In [6]:
ranks = pft.prune(rank_filters=True)

Original Num Filter: 2752


In [7]:
layers_pruned = {}
for layer_index, filter_index in ranks[1]:
    layers_pruned[layer_index] = layers_pruned.get(layer_index, 0) + 1

In [8]:
layers_pruned

{0: 64, 3: 128, 6: 256, 8: 256, 11: 512, 13: 512, 16: 512, 18: 512}

In [9]:
del pft

PruningFineTuner object deleted and memory cleared.


In [10]:
model = SVCNN('svcnn')
model.load_state_dict(torch.load('./model-00030.pth', map_location='mps'))
model = model.to('mps')

In [11]:
pft = PruningFineTuner(model, train_amt=0.01, test_amt=0.1)

In [12]:
prune_amount=0.4
num_filter_to_remove=0.9*pft.total_num_filters()
prune_targets = ranks[1][:int(num_filter_to_remove)]

In [13]:
len(prune_targets)

2476

In [14]:
pruner = Pruning(model)

In [15]:
for layer_index, filter_index in prune_targets:
    model = pruner.prune_conv_layers(model, layer_index=layer_index, filter_index=filter_index)

/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.13/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [16]:
pft.get_model_size(model)

275.8605537414551

In [17]:
mvcnntrainer.get_size(model), mvcnntrainer.get_num_filters(model)

(275.8605537414551, 276)

In [ ]:
# exit()